In [1]:
import torch
torch.nn.Module.dump_patches = True
import os
import shutil
import random
import time
import cv2
import numpy as np
from yacs.config import CfgNode as CN
import sys
import torch
import copy
import queue
import glob
import math
import random
import itertools
import threading
from scipy.spatial.distance import pdist

In [2]:
from utils.datasets import LoadStreams,LoadImages
from utils.utils import torch_utils,google_utils,non_max_suppression,Path,scale_coords,plot_one_box,platform,xyxy2xywh

In [3]:
class KalmanFilter(object):
    def __init__(self,bbox):
        (x,y,w,h)=bbox
        self.x,self.y=0,0
        self.last_measurement = self.current_measurement = np.array((2,1),np.float32)
        np_bbox=np.asarray([x,y]).astype(np.float32)
        self.last_predicition = self.current_prediction = np_bbox.resize((2,1))#np.zeros((2,1),np.float32)
        self.kalman = cv2.KalmanFilter(4, 2)
        #设置测量矩阵
        self.kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]],np.float32)
        #设置转移矩阵
        self.kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]],np.float32)
        #设置过程噪声协方差矩阵
        self.kalman.processNoiseCov = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],np.float32)*0.003
    def move(self,x,y):
        #初始化
        self.last_measurement = self.current_measurement
        self.last_prediction = self.current_prediction
        #传递当前测量坐标值
        self.current_measurement = np.array([[np.float32(x)],[np.float32(y)]])
        #用来修正卡尔曼滤波的预测结果
        self.kalman.correct(self.current_measurement)
        # 调用kalman这个类的predict方法得到状态的预测值矩阵，用来估算目标位置
        current_prediction = self.kalman.predict()
        #上一次测量值
        lmx,lmy = self.last_measurement[0],self.last_measurement[1]
        #当前测量值
        cmx,cmy = self.current_measurement[0],self.current_measurement[1]
        #上一次预测值
        #lpx,lpy = last_prediction[0],last_prediction[1]
        #当前预测值
        cpx,cpy = current_prediction[0],current_prediction[1]
        return cpx,cpy
    def update(self,bbox):
        (x,y,w,h)=bbox
        self.x,self.y = self.move(x,y)
    def get(self):
        return self.x,self.y

In [4]:
import itertools
import numpy as np
from numpy import random
from scipy.optimize import linear_sum_assignment
# 任务分配类
class TaskAssignment(object):         
    # 全排列方法
    def all_permutation(self, task_matrix):
        number_of_choice = len(task_matrix)
        solutions = []
        values = []
        for each_solution in itertools.permutations(range(number_of_choice)):
            each_solution = list(each_solution)
            solution = []
            value = 0
            for i in range(len(task_matrix)):
                value += task_matrix[i][each_solution[i]]
                solution.append(task_matrix[i][each_solution[i]])
            values.append(value)
            solutions.append(solution)
        min_cost = np.min(values)
        best_solution = solutions[values.index(min_cost)]
        return best_solution

    # 匈牙利方法
    def Hungary(self, task_matrix):
        b = task_matrix.copy()
        # 行和列减0
        for i in range(len(b)):
            row_min = np.min(b[i])
            for j in range(len(b[i])):
                b[i][j] -= row_min
        for i in range(len(b[0])):
            col_min = np.min(b[:, i])
            for j in range(len(b)):
                b[j][i] -= col_min
        line_count = 0
        # 线数目小于矩阵长度时，进行循环
        while (line_count < len(b)):
            line_count = 0
            row_zero_count = []
            col_zero_count = []
            for i in range(len(b)):
                row_zero_count.append(np.sum(b[i] == 0))
            for i in range(len(b[0])):
                col_zero_count.append((np.sum(b[:, i] == 0)))
            # 划线的顺序（分行或列）
            line_order = []
            row_or_col = []
            for i in range(len(b[0]), 0, -1):
                while (i in row_zero_count):
                    line_order.append(row_zero_count.index(i))
                    row_or_col.append(0)
                    row_zero_count[row_zero_count.index(i)] = 0
                while (i in col_zero_count):
                    line_order.append(col_zero_count.index(i))
                    row_or_col.append(1)
                    col_zero_count[col_zero_count.index(i)] = 0
            # 画线覆盖0，并得到行减最小值，列加最小值后的矩阵
            delete_count_of_row = []
            delete_count_of_rol = []
            row_and_col = [i for i in range(len(b))]
            for i in range(len(line_order)):
                if row_or_col[i] == 0:
                    delete_count_of_row.append(line_order[i])
                else:
                    delete_count_of_rol.append(line_order[i])
                c = np.delete(b, delete_count_of_row, axis=0)
                c = np.delete(c, delete_count_of_rol, axis=1)
                line_count = len(delete_count_of_row) + len(delete_count_of_rol)
                # 线数目等于矩阵长度时，跳出
                if line_count == len(b):
                    break
                # 判断是否画线覆盖所有0，若覆盖，进行加减操作
                if 0 not in c:
                    row_sub = list(set(row_and_col) - set(delete_count_of_row))
                    min_value = np.min(c)
                    for i in row_sub:
                        b[i] = b[i] - min_value
                    for i in delete_count_of_rol:
                        b[:, i] = b[:, i] + min_value
                    break
        row_ind, col_ind = linear_sum_assignment(b)
        #min_cost = task_matrix[row_ind, col_ind].sum()
        best_solution = list(task_matrix[row_ind, col_ind])
        return  best_solution

In [5]:
import tensorflow as tf

def IoU_calculator(x, y, w, h, l_x, l_y, l_w, l_h):
    """calaulate IoU
    Args:
      x: net predicted x
      y: net predicted y
      w: net predicted width
      h: net predicted height
      l_x: label x
      l_y: label y
      l_w: label width
      l_h: label height
    
    Returns:
      IoU
    """
    
    # convert to coner
    x_max = x + w/2
    y_max = y + h/2
    x_min = x - w/2
    y_min = y - h/2
 
    l_x_max = l_x + l_w/2
    l_y_max = l_y + l_h/2
    l_x_min = l_x - l_w/2
    l_y_min = l_y - l_h/2
    # calculate the inter
    inter_x_max = tf.minimum(x_max, l_x_max)
    inter_x_min = tf.maximum(x_min, l_x_min)
 
    inter_y_max = tf.minimum(y_max, l_y_max)
    inter_y_min = tf.maximum(y_min, l_y_min)
 
    inter_w = inter_x_max - inter_x_min
    inter_h = inter_y_max - inter_y_min
    
    inter = tf.cond(tf.logical_or(tf.less_equal(inter_w,0), tf.less_equal(inter_h,0)), 
                    lambda:tf.cast(0,tf.float32), 
                    lambda:tf.multiply(inter_w,inter_h))
    # calculate the union
    union = w*h + l_w*l_h - inter
    
    IoU = inter / union
    return IoU
#交集占第一个bbox的面积比例
def Area_iou(x, y, w, h, l_x, l_y, l_w, l_h):
    """calaulate IoU
    Args:
      x: net predicted x
      y: net predicted y
      w: net predicted widthA
      h: net predicted height
      l_x: label x
      l_y: label y
      l_w: label width
      l_h: label height
    
    Returns:
      IoU
    """
    
    # convert to coner
    x_max = x + w/2
    y_max = y + h/2
    x_min = x - w/2
    y_min = y - h/2
 
    l_x_max = l_x + l_w/2
    l_y_max = l_y + l_h/2
    l_x_min = l_x - l_w/2
    l_y_min = l_y - l_h/2
    # calculate the inter
    inter_x_max = tf.minimum(x_max, l_x_max)
    inter_x_min = tf.maximum(x_min, l_x_min)
 
    inter_y_max = tf.minimum(y_max, l_y_max)
    inter_y_min = tf.maximum(y_min, l_y_min)
 
    inter_w = inter_x_max - inter_x_min
    inter_h = inter_y_max - inter_y_min
    
    inter = tf.cond(tf.logical_or(tf.less_equal(inter_w,0), tf.less_equal(inter_h,0)), 
                    lambda:tf.cast(0,tf.float32), 
                    lambda:tf.multiply(inter_w,inter_h))
    # calculate the union
    union = w*h 
    IoU = inter / union
    return IoU

In [6]:
from extractor.feature_extractor import Extractor
class CNN_deepsort:
    def __init__(self):
        self.extractor = Extractor("./extractor/checkpoint/ckpt.t7", use_cuda=True)
    def compute_feature(self,im):
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        #print(im.shape)
        feature = self.extractor([im])
        #print(feature.shape)
        return feature

In [7]:
from torchvision.models.segmentation import fcn_resnet101
from PIL import Image
from torchvision import transforms
from scipy.optimize import linear_sum_assignment
from collections import defaultdict
class CNNREID:
    def __init__(self):
        self.frame=0
        self.color_old=0
        self.flag_frame=[]
        self.flag_line=[]
        self.add_names=[]
        self.dth = 0.01#相似度距离0.01~0.02  / 欧氏距离25
        self.reid_dth=0.02 #分辨不同人的阈值
        self.Forget=10  #状态切换帧数
        #匈牙利算法
        self.task = TaskAssignment()
        #CNN的feature extractor 
        self.CNN = CNN_deepsort()
        #LIB保存
        self.save={'000':{'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'cross':False,'overlap':False,'hold':False,'overlap_id':[]}}
    #*
    #*
    #*      计算集合
    #*
    #*
    
    #计算 余弦距离
    def compute_cos(self,x,y):
        x_,y_=x.flatten(),y.flatten()
        dist =1- abs(np.dot(x_,y_)/(np.linalg.norm(x_)*np.linalg.norm(y_)))      
        return abs(dist)
    #计算 欧氏距离
    def compute_dis(self,x,y):
         return np.linalg.norm( x - y )
    
     #计算两点近距离公式 xyxy
    def distEclud(self,veA,vecA,veB,vecB):
        lossA=veB-veA
        lossB=vecB-vecA
        return math.sqrt(pow(lossA,2)+pow(lossB,2))
    
     #输入图片计算，输出Feature
    def compute_feature_cnn(self,im):
        feature = self.CNN.compute_feature(im)
        return feature
    
    #找出list中same values
    def list_duplicates(self,seq):
        tally = defaultdict(list)
        for i,item in enumerate(seq):
            tally[item].append(i)
        return ((key,locs) for key,locs in tally.items() 
                                if len(locs)>1)
    def compute_IOU(self,bbox1,bbox2):
        (x1,y1,w1,h1)=bbox1
        (x2,y2,w2,h2)=bbox2
        iou = IoU_calculator(x1, y1, w1, h1,x2, y2, w2, h2)
        return float(iou)
    #*                  
    #  *  ID管理函数集   
    #*                  
    
    def clear_id(self):
        self.save={'000':{'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'cross':False,'overlap':False,'hold':False,'overlap_id':[]}}  

    def manage_lib(self,frame):
        for k,v in list(self.save.items()):
            if k=='000' or v['overlap']==True:#or v['passage']==False:
                continue
            lib = v['lib']
            frame_lib=v['frame']
            if frame-frame_lib>self.Forget:
                self.save[k]['frame']=frame
                #self.save[k]['lib'].pop(0)
                if len(lib)<10:
                    self.dele_lib_id(k)
                if  k not in self.save:
                    continue
                if self.save[k]['passage']==True:
                    center =self.save[k]['center']
                    for lib_ in self.save[k]['lib']:
                        center=center*0.9+lib_*0.1
                    #self.save[k]['lib'].pop(0)
                    self.passage_off(k,center)
                    #self.flag_frame=[]
    def hold_on(self,id_):
        self.save[id_]['hold']=True
    def hold_off(self):    
        for id_ in self.save.keys():
            self.save[id_]['hold']=False
        
    def paassag_on(self,feature,bbox,frame,id_):
        self.save[id_]['center']=feature
        self.save[id_]['bbox']=bbox
        self.save[id_]['lib'].append(feature)
        self.save[id_]['frame']=frame
        self.save[id_]['passage']=True
        print('%s passage is True'%id_)
        
    def passage_off(self,id_,center):
        self.save[id_]['center']=center
        self.save[id_]['passage']=False
        print('%s passage is False'%id_)
    
    def overlap_on(self,id_):
        self.save[id_]['overlap']=True
        #print('%s overlap is True'%id_)
        
    def overlap_off(self,id_):
        self.save[id_]['overlap']=False
        #print('%s overlap is False'%id_)
        
    def cross_on(self,id_):
        self.save[id_]['cross']=True
        #print('%s cross is True'%id_)
        
    def cross_off(self,id_):
        self.save[id_]['cross']=False
        #print('%s cross is False'%id_)
        
    def update_frame(self,id_,frame):
        self.save[id_]['frame']=frame
        
    def dele_lib_id(self,id_):
        del self.save[id_]
        print('the id %s Remove'%id_)
        
    def add_lib(self,feature,bbox,frame,passage=False):
        save={'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'cross':False,'overlap':False,'hold':False,'overlap_id':[]}
        for n in  self.save.keys():
            id_=n
        self.id_new=('%03d'%(int(id_)+1))
        save['center']=feature
        save['bbox']=bbox
        save['lib']=[feature]
        save['frame']=frame
        save['passage']=passage
        self.save[self.id_new]=save
        print('find id %s'%self.id_new)
        return self.id_new
    
    def add_lib_overlab(self,feature,bbox,frame,list_ids):
        save={'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'cross':False,'overlap':False,'hold':False,'overlap_id':[]}
        for n in  self.save.keys():
            id_=n
        save['overlap']=True
        self.id_new=('%03d'%(int(id_)+1))
        save['overlap_id']=list_ids
        save['center']=feature
        save['bbox']=bbox
        save['lib']=[feature]
        save['hold']=True
        save['frame']=frame
        save['passage']=True
        self.save[self.id_new]=save
        print('find overlap id %s'%self.id_new)
        return self.id_new
    
    def check_overlap_id(self,bbox):
        list_overlap_ids,list_dis,list_overlap_name=[],[],[]
        for k,v in self.save.items():
            if k!='000' and v['passage']==True and v['cross']==True:
                dis = self.distEclud(bbox[0],bbox[1],v['bbox'][0],v['bbox'][1])
                list_overlap_name.append(k)
                list_dis.append(dis)
        #print(list_overlap_name,list_dis)
        list_dis_copy = copy.copy(list_dis)
        if len(list_dis)>=2:
            for i in range(2):
                list_overlap_ids.append(list_overlap_name[list_dis.index(min(list_dis_copy))])
                list_dis_copy.pop(list_dis_copy.index(min(list_dis_copy)))
        return list_overlap_ids
    
    def overlap_reid(self,featureVector,bbox,frame):
        color_=0
        over_id=''
        list_feature_distance,list_name_overlap=[],[]
        id_ = self.add_id(featureVector,bbox,frame,state='overlap')
        if id_!=' ':
            color_ = self.check_lib_add(featureVector,id_,bbox)
            ids = self.save[id_]['overlap_id']
            for id_ in ids:
                self.save[id_]['frame']=self.frame
                self.hold_on(id_)
                over_id += ('&%s'%id_)
        else:
            over_id=id_
        return over_id,color_    
        
        
    def add_id(self,feature,bbox,frame,state='None'):   
        #print(len(self.save))
        if frame!=0 and len(self.save)!=1:
            if state=='None':
                id_ = self.check_lib_passage(feature)
                #self.id_new = self.check_lib(feature)
                if id_ == ' ':
                    self.flag_line=[]
                    self.add_names=[]
                    if self.id_new in self.save:
                        if frame-self.save[self.id_new]['frame']>1:
                            self.flag_frame=[]
                    else:
                        self.flag_frame=[]
                    if len(self.flag_frame)==0:
                        self.id_new = self.add_lib(feature,bbox,frame)
                    self.flag_frame.append(frame)
                    self.save[self.id_new]['frame']=frame
                    #self.update_frame
                    if len(self.flag_frame)>=5:
                        if np.mean(self.flag_frame)==self.flag_frame[2]:
                            self.paassag_on(feature,bbox,frame,self.id_new)
                            self.flag_frame=[]
                        else:
                            self.dele_lib_id(self.id_new)
                            self.flag_frame=[]
                else:
                    #if frame - self.save[id_]['frame'] >3:
                    self.flag_frame=[]
                    #self.add_names.append(id_)
                    self.flag_line.append(frame)
                    self.save[id_]['frame']=frame
                    #self.update_frame 
                    if len(self.flag_line)>=3:
                        if np.mean(self.flag_line)==self.flag_line[1]:
                            self.paassag_on(feature,bbox,frame,id_)
                            self.flag_line=[]
                        else:
                            self.flag_line=[]
                    return id_
            if state=='overlap':
                list_name_overlap,list_feature_distance=[],[]
                for k,v in self.save.items():
                    list_=[]
                    if k!='000' and v['overlap']==True:
                        for feature_ in v['lib']:
                            list_.append(self.compute_cos(feature,feature_))
                        list_name_overlap.append(k)
                        list_feature_distance.append(min(list_))
                if len(list_name_overlap)==0:
                    list_ids = self.check_overlap_id(bbox)
                    if len(list_ids)>=2:
                        self.id_new = self.add_lib_overlab(feature,bbox,frame,list_ids)
                    else:
                        self.id_new=' '
                elif min(list_feature_distance)<self.reid_dth:
                    self.id_new = list_name_overlap[list_feature_distance.index(min(list_feature_distance))]  
                else:
                    list_ids = self.check_overlap_id(bbox)
                    if len(list_ids)>=2:
                        self.id_new = self.add_lib_overlab(feature,bbox,frame,list_ids)
                    else:
                        self.id_new=' '
        else:
            self.id_new = self.add_lib(feature,bbox,frame,passage=True)
        return self.id_new
    
#*                  
#*      检测函数集   
#*                  
                
    def check_frame_id(self,id_,feature,frame):
        list_dis=[]
        for k,v in list(self.save.items()):
            if k==id_:
                if frame-(v['frame'])>=10:
                        if k==id_:
                            for lib_ in v['lib']:
                                list_dis.append(self.compute_cos(lib_,feature))
                            if min(list_dis)<self.dth:
                                return True
                            else:
                                return False
                else:
                    return True
        return False
    #去除比例不好的bbox防止发生错误，去除进入出去的bbox
    def check_bboxs_select(self,bboxs):
        for bbox in bboxs:
            (x,y,w,h)=bbox
            if w/h <0.2:
                bboxs.remove(bbox)
        return bboxs
   #检查bbox是否有交叉
    def check_bboxs_cross(self,bboxs,bbox):    
        ret=False
        (xa, ya, wa, ha)=bbox
        cross_bboxs=copy.copy(bboxs)
        cross_bboxs.remove(bbox)
        for bbox_ in cross_bboxs:
            (xb, yb, wb, hb) = bbox_ 
            distance = self.distEclud(xa,ya,xb,yb)
            if (distance/(wa+wb))>=0.5:
                continue
            if (distance/(wa+wb))<0.5:
                ret=True  
        return ret 
    #检查bbox是否有重叠
    def check_bboxs_overlap(self,bbox):
        ret=False  
        flag_num=0
        (xc, yc, wc, hc)=bbox
        for k,v in list(self.save.items()):
            if k!='000'and v['passage']==True and v['cross']==True and v['overlap']==False:
                (xl, yl, wl, hl) = v['bbox']
                
                distance = self.distEclud(xc,yc,xl,yl)
                iou = IoU_calculator(xc, yc, wc, hc,xl, yl, wl, hl)
                if (distance/(wc+wl))>=0.5:
                    continue
                if (distance/(wc+wl))<0.5 and iou>0.1:  
                    flag_num+=1
        if flag_num>1:
            ret=True
        return ret
    def check_lib_add(self,featureVector,min_distance_id,bbox):
        for k,v in list(self.save.items()):
            list_dis=[]
            if  k==min_distance_id:
                n=v['center']
                lib = v['lib']
                for lib_ in lib:
                    list_dis.append(self.compute_cos(lib_,featureVector))
                if len(self.save[min_distance_id]['lib'])>511:
                    num = list_dis.index(min(list_dis))
                    self.save[min_distance_id]['lib'].pop(num)
                    #self.save[min_distance_id]['lib'].pop(0)
                if len(list_dis)>0 and np.mean(list_dis)>self.dth:
                    self.save[min_distance_id]['lib'].append(featureVector)
                    print('\r%s lib len :%d'%(min_distance_id,len(lib)),end='')
                break
        return int(len(lib)/2)
    #检查就绪的
    def check_lib(self,featureVector):
        list_id,list_mean=[],[]
        for k,v in list(self.save.items()):
            list_dis=[]
            if k=='000'or v['passage']==False :
                continue
            lib = v['lib']
            for lib_ in lib:
                list_dis.append(self.compute_cos(lib_,featureVector))
            list_id.append(k)
            list_mean.append(min(list_dis)) 
        if len(list_mean)>0 and min(list_mean) <self.dth:
            return list_id[list_mean.index(min(list_mean))] 
        else:
            #print( min(list_mean))
            return ' '   
    #检查未就绪
    def check_lib_passage(self,featureVector):
        list_id,list_mean=[],[]
        for k,v in list(self.save.items()):
            list_dis=[]
            if k!='000' and v['passage']==False and len(v['lib'])>10:
                lib = v['lib']
                for lib_ in lib:
                    list_dis.append(self.compute_cos(lib_,featureVector))
                list_id.append(k)
                list_mean.append(min(list_dis)) 
        if len(list_mean)>0 and min(list_mean) <self.reid_dth:
            #print(list_mean)
            return list_id[list_mean.index(min(list_mean))] 
        else:
            #print( min(list_mean))
            #if len(list_mean)>0:
                #print(list_mean)
            return ' '      
    def check_from_overlap(self,bbox):
        ret=False
        for k,v in self.save.items():
            if v['overlap']==True:
                iou = self.compute_IOU(bbox,v['bbox'])
                num = len(v['overlap_id'])
                if iou>0:
                    ret=True
                    #print(self.frame,iou)
        return ret
    
    #*****************
    #
    #   四大模式判断
    #
    #
    #****************
    
    def None_mode(self,notebook,im):
        CNN_bbox,list_id,list_feature,list_distance,list_center,list_c=[],[],[],[],[],[]
        for bbox,c in notebook.items():
            CNN_bbox.append(bbox)
            (x, y, w, h) = bbox
            #print(bbox)
            (x1, y1, x2, y2)=abs(int(x-w/2)), abs(int(y-h/2)), int(x+w/2), int(y+h/2)
            #print((x1, y1, x2, y2))
            im_=im[y1:y2,x1:x2]
            im_ = cv2.resize(im_,(64,128))
            #featureVector = self.compute_feature(im_)
            featureVector = self.compute_feature_cnn(im_)
            #不交叉不重叠
            distance=0
            #所有图片与LIB中passage为TRUE的计算距离
            list_distance_buffer,list_id_buffer,list_center_buffer=[],[],[]
            for k,v in list(self.save.items()):
                if k=='000':
                    continue
                if v['passage']==True and v['hold']==False and v['overlap']==False:
                    n=v['center']
                    #print(distance_kalman)
                    distance_feature = self.compute_cos(n,featureVector)
                    #distance =+distance_feature*0.95+distance_kalman*0.005
                    list_distance_buffer.append(distance_feature)
                    list_id_buffer.append(k)
                    list_center_buffer.append(n)
            if len(list_distance_buffer)>0 :
                list_distance.append(list_distance_buffer)
                list_id.append(list_id_buffer)
                list_center.append(list_center_buffer)
                list_feature.append(featureVector)
                list_c.append(c)
            else:
                #ret = self.check_bboxs(bbox_list,bbox) 
                #if ret ==True:
                id_ = self.add_id(featureVector,bbox,self.frame)
                self.lecture[bbox]=[id_,c,0]
        #匈牙利算法
        list_bbox_loss,list_feature_loss,confidences_loss=[],[],[]
        if CNN_bbox!=[] and list_distance!=[]:
                CNN_center,CNN_feature,CNN_id,CNN_c=[],[],[],[]
                #需要id的数量要小于等于就绪id的数量
                if len(CNN_bbox)>len(list_distance[0]):
                    min_distance=[]
                    for id_distanc in list_distance:
                        min_distance.append(min(id_distanc))
                    for i in range((len(CNN_bbox)-len(list_distance[0]))):
                        if len(min_distance)==0:
                            continue
                        num=min_distance.index(max(min_distance))
                        min_distance.pop(num)
                        list_distance.pop(num)
                        list_id.pop(num)
                        list_center.pop(num)
                        list_feature_loss.append(list_feature.pop(num))
                        list_bbox_loss.append(CNN_bbox.pop(num))
                        confidences_loss.append(list_c.pop(num))
                if len(list_distance)>0:
                    np_distance=np.asarray(list_distance)
                    if np_distance.shape[0]<2:
                        list_Hungary=[min(np_distance.flatten().tolist())]
                    else:
                        list_Hungary = self.task.Hungary(np_distance)
                    num_CNN=0    
                    for number in list_Hungary:
                        #number = number*0.00001               
                        min_distance = np_distance[num_CNN].flatten().tolist().index(number)
                        CNN_id.append(list_id[num_CNN][min_distance])
                        CNN_center.append(list_center[num_CNN][min_distance])
                        CNN_feature.append(list_feature[num_CNN])
                        CNN_c.append(list_c[num_CNN])
                        num_CNN+=1
                for bbox,center,feature,id_,c in zip(CNN_bbox,CNN_center,CNN_feature,CNN_id,CNN_c):
                     #检查除id_me外是否与数据库里的bbox重叠
                    center_ = (center*0.5)+(feature*0.5)
                    self.save[id_]['center'] =center_
                    self.save[id_]['bbox'] =bbox
                    self.save[id_]['frame'] =self.frame
                    #print(bbox[2]*bbox[3],self.frame)
                    color_=self.check_lib_add(feature,id_,bbox)
                    self.lecture[bbox]=[id_,c,color_]
                    self.hold_on(id_)

                #没有ID,重新add_id    
                #print(list_bbox_loss,list_feature_loss,confidences)
                for bbox,featureVector,c in zip(list_bbox_loss,list_feature_loss,confidences_loss):
                    #print(bbox,featureVector,c)
                    id_ = self.add_id(featureVector,bbox,self.frame)
                    self.lecture[bbox]=[id_,c,0]

    def Overlap_mode(self,notebook,im):
        for bbox,c in notebook.items():
            (x, y, w, h) = bbox
            #print(bbox)
            (x1, y1, x2, y2)=abs(int(x-w/2)), abs(int(y-h/2)), int(x+w/2), int(y+h/2)
            #print((x1, y1, x2, y2))
            im_=im[y1:y2,x1:x2]
            im_ = cv2.resize(im_,(64,128))
            #featureVector = self.compute_feature(im_)
            featureVector = self.compute_feature_cnn(im_)
            id_,color_= self.overlap_reid(featureVector,bbox,self.frame)
            self.lecture[bbox]=[id_,c,color_]
            
    def Cross_from_overlap_mode(self,notebook,im):  
        cross_from_overlap_bboxs,cross_from_overlap_cs,cross_from_overlap_ids,cross_from_overlap_features,cross_from_overlap_distances,cross_from_overlap_distance_buffer=[],[],[],[],[],[]
        del_ids=[]
        notebook_to_cross={}
        for bbox,c in notebook.items():
            (x, y, w, h) = bbox
            (x1, y1, x2, y2)=abs(int(x-w/2)), abs(int(y-h/2)), int(x+w/2), int(y+h/2)
            im_=im[y1:y2,x1:x2]
            im_ = cv2.resize(im_,(64,128))
            featureVector = self.compute_feature_cnn(im_)
            cross_from_overlap_bboxs.append(bbox)
            cross_from_overlap_cs.append(c)
            cross_from_overlap_features.append(featureVector)
        if len(cross_from_overlap_bboxs)>0:
            for k,v in self.save.items(): 
                if v['overlap']==True:
                    cross_from_overlap_ids+=v['overlap_id'] 
                    del_ids.append(k)
            for feature_ in cross_from_overlap_features:
                cross_from_overlap_distance_buffer=[]
                for id_ in cross_from_overlap_ids:
                    if id_ in self.save.keys():
                        cross_from_overlap_distance_buffer.append(self.compute_cos(self.save[id_]['center'],feature_))
                    else:
                        cross_from_overlap_ids.remove(id_)
                cross_from_overlap_distances.append(cross_from_overlap_distance_buffer)
            if len(cross_from_overlap_bboxs)>len(cross_from_overlap_ids):
                list_min=[]
                for dis_ in cross_from_overlap_distances:
                    list_min.append(min(dis_))
                for i in range(len(cross_from_overlap_bboxs)-len(cross_from_overlap_ids)):
                    if len(list_min)==0:
                                continue
                    num = list_min.index(max(list_min))
                    #出现多余的bbox移交给cross处理
                    notebook_to_cross[cross_from_overlap_bboxs.pop(num)]=cross_from_overlap_cs.pop(num)
                    cross_from_overlap_distances.pop(num)
                    cross_from_overlap_features.pop(num)
                    list_min.pop(num)
        cross_from_overlap_id,cross_from_overlap_center,cross_from_overlap_feature,cross_from_overlap_c=[],[],[],[]
        #匈牙利算法        
        if len(cross_from_overlap_bboxs)>0:
            np_cross_from_overlap_distances = np.asarray(cross_from_overlap_distances)
            if np_cross_from_overlap_distances.shape[0]<2:
                list_Hungary=[min(np_cross_from_overlap_distances.flatten().tolist())]
            else:
                list_Hungary = self.task.Hungary(np_cross_from_overlap_distances)
            num_CNN=0    
            #print(self.frame,np_cross_from_overlap_distances,list_Hungary)
            for number in list_Hungary:              
                min_distance = np_cross_from_overlap_distances[num_CNN].flatten().tolist().index(number)
                cross_from_overlap_id.append(cross_from_overlap_ids[num_CNN])
                cross_from_overlap_feature.append(cross_from_overlap_features[num_CNN])
                cross_from_overlap_c.append(cross_from_overlap_cs[num_CNN])
                num_CNN+=1 
        for bbox,feature,id_,c in zip(cross_from_overlap_bboxs,cross_from_overlap_feature,cross_from_overlap_id,cross_from_overlap_c):
                center  = self.save[id_]['center']
                center_ = (center*0.5)+(feature*0.5)
                self.save[id_]['center'] =center_
                self.save[id_]['bbox'] =bbox
                self.save[id_]['frame'] =self.frame
                #print(bbox[2]*bbox[3],self.frame)
                color_=self.check_lib_add(feature,id_,bbox)
                self.lecture[bbox]=[id_,c,0]
                self.hold_on(id_)
                self.cross_on(id_)
                self.overlap_off(id_)
        for id_ in del_ids:
            self.dele_lib_id(id_)
        self.Cross_mode(notebook_to_cross,im)
            
    def Cross_mode(self,notebook,im):     
        CNN_bbox,list_id,list_feature,list_distance,list_center,list_c=[],[],[],[],[],[]
        for bbox,c in notebook.items():
            CNN_bbox.append(bbox)
            (x, y, w, h) = bbox
            #print(bbox)
            (x1, y1, x2, y2)=abs(int(x-w/2)), abs(int(y-h/2)), int(x+w/2), int(y+h/2)
            #print((x1, y1, x2, y2))
            im_=im[y1:y2,x1:x2]
            im_ = cv2.resize(im_,(64,128))
            #featureVector = self.compute_feature(im_)
            featureVector = self.compute_feature_cnn(im_)
            #不交叉不重叠
            distance=0
            #所有图片与LIB中passage为TRUE的计算距离
            list_distance_buffer,list_id_buffer,list_center_buffer=[],[],[]
            for k,v in list(self.save.items()):
                if k=='000':
                    continue
                if v['passage']==True and v['hold']==False and v['overlap']==False:
                    n=v['center']
                    #print(distance_kalman)
                    distance_feature = self.compute_cos(n,featureVector)
                    #distance =+distance_feature*0.95+distance_kalman*0.005
                    list_distance_buffer.append(distance_feature)
                    list_id_buffer.append(k)
                    list_center_buffer.append(n)
            if len(list_distance_buffer)>0 :
                list_distance.append(list_distance_buffer)
                list_id.append(list_id_buffer)
                list_center.append(list_center_buffer)
                list_feature.append(featureVector)
                list_c.append(c)
            else:
                #ret = self.check_bboxs(bbox_list,bbox) 
                #if ret ==True:
                id_ = self.add_id(featureVector,bbox,self.frame)
                lecture[bbox]=id_
                dths.append(c)
                color.append(0) 
        #匈牙利算法
        list_bbox_loss,list_feature_loss,confidences_loss=[],[],[]
        if CNN_bbox!=[] and list_distance!=[]:
                CNN_center,CNN_feature,CNN_id,CNN_i,CNN_c=[],[],[],[],[]
                #需要id的数量要小于等于就绪id的数量
                if len(CNN_bbox)>len(list_distance[0]):
                    min_distance=[]
                    for id_distanc in list_distance:
                        min_distance.append(min(id_distanc))
                    for i in range((len(CNN_bbox)-len(list_distance[0]))):
                        if len(min_distance)==0:
                            continue
                        num=min_distance.index(max(min_distance))
                        min_distance.pop(num)
                        list_distance.pop(num)
                        list_id.pop(num)
                        list_center.pop(num)
                        list_feature_loss.append(list_feature.pop(num))
                        list_bbox_loss.append(CNN_bbox.pop(num))
                        confidences_loss.append(list_c.pop(num))
                if len(list_distance)>0:
                    np_distance=np.asarray(list_distance)
                    if np_distance.shape[0]<2:
                        list_Hungary=[min(np_distance.flatten().tolist())]
                    else:
                        list_Hungary = self.task.Hungary(np_distance)
                    num_CNN=0    
                    for number in list_Hungary:
                        #number = number*0.00001               
                        min_distance = np_distance[num_CNN].flatten().tolist().index(number)
                        CNN_id.append(list_id[num_CNN][min_distance])
                        CNN_center.append(list_center[num_CNN][min_distance])
                        CNN_feature.append(list_feature[num_CNN])
                        CNN_c.append(list_c[num_CNN])
                        num_CNN+=1
                for bbox,center,feature,id_,c in zip(CNN_bbox,CNN_center,CNN_feature,CNN_id,CNN_c):
                     #检查除id_me外是否与数据库里的bbox重叠
                    center_ = (center*0.5)+(feature*0.5)
                    self.save[id_]['center'] =center_
                    self.save[id_]['bbox'] =bbox
                    self.save[id_]['frame'] =self.frame
                    #print(bbox[2]*bbox[3],self.frame)
                    color_=self.check_lib_add(feature,id_,bbox)
                    self.lecture[bbox]=[id_,c,color_]
                    self.hold_on(id_)
                    self.cross_on(id_)
                    self.overlap_off(id_)
                #没有ID,重新add_id    
                #print(list_bbox_loss,list_feature_loss,confidences)
                for bbox,featureVector,c in zip(list_bbox_loss,list_feature_loss,confidences_loss):
                    #print(bbox,featureVector,c)
                    id_ = self.add_id(featureVector,bbox,self.frame)
                    #notebook[bbox]=id_
                    self.lecture[bbox]=[id_,c,0]
      #主要函数
    def cross_video_pca(self,bbox_list,confidences,im):
        #
        # * self.notebook 记录发现新的图片的W
        # * 保存数据的格式 self.save={'id':{'center':w,'LIB':[w1..wn]}}
        #
        color,dths,identities=[],[],[]
        self.lecture={}
        #维护LIB
        self.manage_lib(self.frame)
        if bbox_list!=[]:
            bbox_list = self.check_bboxs_select(bbox_list)
            #四个状态存储
            none_notebook,cross_notebook,overlap_notebook,cross_from_overlap_notebook={},{},{},{}
            for bbox,c in zip(bbox_list,confidences):
                #交叉判断
                #print(bbox_list)
                ret_bbox_cross = self.check_bboxs_cross(bbox_list,bbox)
                #print(ret_bbox_cross,flag)
                #判断是否重叠重叠状态是从交叉到重叠
                ret_bbox_overlap=self.check_bboxs_overlap(bbox)
                #有交叉的时候直接进行Kalkman
                if ret_bbox_cross:
                    ret_cross_fromo_verlap = self.check_from_overlap(bbox)
                    if ret_cross_fromo_verlap:
                        cross_from_overlap_notebook[bbox]=c
                    else:
                        cross_notebook[bbox]=c
                #无交叉判断有重叠
                elif ret_bbox_overlap:
                        overlap_notebook[bbox]=c
                else:
                    ret_cross_fromo_verlap = self.check_from_overlap(bbox)
                    if ret_cross_fromo_verlap:
                        overlap_notebook[bbox]=c
                    else:
                        none_notebook[bbox]=c
                        
            self.None_mode(none_notebook,im)
            self.Overlap_mode(overlap_notebook,im)
            self.Cross_mode(cross_notebook,im)
            self.Cross_from_overlap_mode(cross_from_overlap_notebook,im)
                    
        for bbox in bbox_list:
            if bbox in self.lecture.keys():
                identities.append(self.lecture[bbox][0])
                dths.append(self.lecture[bbox][1])
                color.append(self.lecture[bbox][2])
        self.hold_off()  
        self.frame+=1
        return identities,dths,color,self.frame

In [8]:
def display(bbox_list,im,identities,dths,color,frame,t_):
    w_im,h_im=im.shape[:2]
    #print(len(out_list))
    for bbox,id_,dth,color_ in zip(bbox_list,identities,dths,color):
        #(x1, y1, x2, y2) = bbox
        (x, y, w, h) = bbox
        (x1, y1, x2, y2)=int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
        cv2.rectangle(im, (x1, y1), (x2, y2), (0, color_, 255-color_), 2)
        #cv2.putText(im, ('%.2f'%c), (x2+10, y1+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 250,0), 1)
        cv2.putText(im, ('%s'%id_), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, color_, 255-color_), 2)
        cv2.putText(im, ('%.2f'%dth), (x2, y1+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, color_, 255-color_), 2)
        cv2.circle(im, (int(x),int(y)), 1, (0, 255, 0), 2)
    fps = 1/(time.time()-t_)
    cv2.putText(im, ('frame:%.2f'%frame), (h_im-500, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.putText(im, ('FPS:%.2f'%fps), (h_im-200, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.putText(im, ('Id number: %d'%len(bbox_list)), (h_im-500, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.imshow('V5', im)
    return im
def yolov5_detect(YOLOV5_CFG,pca):
    device = torch_utils.select_device(YOLOV5_CFG.device)
    model = torch.load(YOLOV5_CFG.weights, map_location=device)['model']
    #model = torch.load(YOLOV5_CFG.weights, map_location=device)
    model.to(device).eval()
    names = model.names if hasattr(model, 'names') else model.modules.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]
    t=0
    if os.path.isfile(YOLOV5_CFG.source) is True:
        cap = cv2.VideoCapture(YOLOV5_CFG.source)
        w0,h0,fps,max_index = int(cap.get(3)),int(cap.get(4)),int(cap.get(5)),int(cap.get(7))
        '''这里直接不缩放直接通过设定的w计算h'''
        w1 = YOLOV5_CFG.im_size[0]
        h1 = h0/(w0/YOLOV5_CFG.im_size[0])
        h1 = int((h1//32))*32
        if YOLOV5_CFG.output is not False:
            fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
            out = cv2.VideoWriter(YOLOV5_CFG.output, fourcc, fps, (w0,h0))
        for i in range(max_index):
            re,im = cap.read()
            net_input = np.transpose(cv2.resize(im, (w1,h1))/255,(2,0,1)).reshape((-1,3,h1,w1))
            net_input = torch.from_numpy(net_input).to(device).type(torch.float32)
            t = time.time()
            pred = model(net_input, augment=YOLOV5_CFG.augment)[0]
            pred = non_max_suppression(pred, YOLOV5_CFG.conf_thres, YOLOV5_CFG.iou_thres, fast=True, classes=YOLOV5_CFG.classes, agnostic=YOLOV5_CFG.agnostic_nms)
            if pred is None or pred[0] is None:
                cv2.imshow('V5',im)
                if YOLOV5_CFG.output is not False:
                    out.write(im)
                continue
            w_im,h_im=im.shape[:2]
            #print(w_im,h_im)
            area_im=int(w_im*h_im)
            bboxes,confs,cats = pred[0][:,:4].cpu().detach().numpy(),pred[0][:,4].cpu().detach().numpy(),pred[0][:,5].cpu().detach().numpy()
            bboxes[:,[0,2]],bboxes[:,[1,3]]=bboxes[:,[0,2]]*(w0/w1),bboxes[:,[1,3]]*(h0/h1)
            list_bbox,list_conf=[],[]
            for bbox,conf,cat in zip(bboxes.astype(np.int),confs,cats.astype(np.int)):
                if (names[cat]=='person'):
                    #p_min,p_max = (bbox[0],bbox[1]),(bbox[2],bbox[3])
                    #im = cv2.rectangle(im, p_min, p_max, (255,0,123), 1, cv2.LINE_AA)
                    #im = cv2.putText(im, '%s %.2f'%(names[cat],conf), p_min, cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
                    (x1, y1, x2, y2)=bbox
                    bbox_=(int(abs((x1+x2)*0.5)),int(abs((y1+y2)*0.5)),int(abs(x1-x2)),int(abs(y1-y2)))
                    (x,y,w,h)=bbox_
                    area=int(w*h)
                    if (area/area_im) <0.01:
                        continue
                    list_bbox.append(bbox_)
                    list_conf.append(conf)
            #CrossVideo Reid
            identities,dths,color,frame = pca.cross_video_pca(list_bbox,list_conf,im)
            display(list_bbox,im,identities,dths,color,frame,t)
            t=time.time()
            if YOLOV5_CFG.output is not False:
                out.write(im)
            if cv2.waitKey(1)&0xff==ord('q'):
                break
        cap.release()
        if YOLOV5_CFG.output is not False:
            out.release()
        cv2.destroyAllWindows()

In [9]:
YOLOV5_CFG = CN()
YOLOV5_CFG.agnostic_nms = False
YOLOV5_CFG.augment      = False
YOLOV5_CFG.classes      = False
YOLOV5_CFG.device       = '0'

YOLOV5_CFG.weights = 'weights/yolov5l.pt'
YOLOV5_CFG.source = './testvideo/testkalman.mp4'
name = time.strftime('%Y.%m.%d',time.localtime(time.time()))
YOLOV5_CFG.output = './savevideo/'+name+'.mp4'


YOLOV5_CFG.save_npz     = False
YOLOV5_CFG.conf_thres   = 0.3
YOLOV5_CFG.iou_thres    = 0.3
YOLOV5_CFG.im_size      = [640,512]
YOLOV5_CFG.freeze()

Reid=CNNREID()
yolov5_detect(YOLOV5_CFG,Reid)

Using CUDA device0 _CudaDeviceProperties(name='GeForce GTX 1080 Ti', total_memory=11264MB)



d:\programs\miniconda3\envs\reid\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'models.yolo.Model' has changed. Tried to save a patch, but couldn't create a writable file Model.patch. Make sure it doesn't exist and your working directory is writable.
  warnings.warn(msg, SourceChangeWarning)


find id 001
001 lib len :8find id 002
001 lib len :12002 passage is True
002 lib len :198find overlap id 003
002 lib len :199the id 003 Remove
001 lib len :278find overlap id 003
001 lib len :279the id 003 Remove
002 lib len :319

KeyboardInterrupt: 